## Guppy v6 Duplex basecalling

* Re-Basecalling using guppy_v6
* comparing guppyv5 to guppyv6 output using PycoQC and MinIONQC (Should use nanoplot next time)

In [1]:
WKDIR=/workspace/hraijc/BB_M7_Assembly

In [2]:
cd /workspace/hraijc/BB_M7_Assembly

In [3]:
mkdir log

mkdir: cannot create directory ‘log’: File exists


: 1

In [1]:
module load pfr-python3

In [2]:
#Need to add the guppy basecaller to the path, not sure how to get around this.
export PATH="/home/hraijc/.local/bin:/workspace/hraijc/packages/guppyV6/ont-guppy/bin:$PATH"

In [3]:
/home/hraijc/.local/bin/guppy_duplex -h

usage: guppy_duplex [-h] [--basecaller_exe BASECALLER_EXE]
                    [--duplex_basecaller_exe DUPLEX_BASECALLER_EXE]
                    --input_path INPUT_PATH --save_path SAVE_PATH
                    [--disable_logging] [--skip_simplex] [--skip_duplex]
                    [--call_non_duplex_reads] [--device DEVICE]
                    [--do_read_splitting]
                    [--duplex_chunks_per_runner DUPLEX_CHUNKS_PER_RUNNER]
                    [--recursive]

All-in-one processing of r10.4 duplex reads using guppy

optional arguments:
  -h, --help            show this help message and exit
  --basecaller_exe BASECALLER_EXE
                        Path to the guppy_basecaller executable. If
                        unspecified it is assumed that the basecaller exe is
                        in the path.
  --duplex_basecaller_exe DUPLEX_BASECALLER_EXE
                        Path to the guppy_basecaller_duplex executable. If
                        unspecified it is assum

In [3]:
bsub -R 'select[defined(gpu)]' -I "/workspace/hraijc/packages/guppyV6/ont-guppy/bin/guppy_basecaller --print_workflows" 

Job <738459> is submitted to default queue <lowpriority>.
<<Waiting for dispatch ...>>
<<Starting on wkoppg33>>
Loading model version information, please wait .................................
Available flowcell + kit combinations are:
flowcell       kit               barcoding config_name                    model version
FLO-PRO001     SQK-RNA002                  rna_r9.4.1_70bps_hac_prom      2020-09-21_rna_r9.4.1_promethion_256_855130ab
FLO-PRO002     SQK-RNA002                  rna_r9.4.1_70bps_hac_prom      2020-09-21_rna_r9.4.1_promethion_256_855130ab
FLO-PRO002-ECO SQK-RNA002                  rna_r9.4.1_70bps_hac_prom      2020-09-21_rna_r9.4.1_promethion_256_855130ab
FLO-PRO002     SQK-LSK112                  dna_r9.4.1_e8.1_hac_prom       2021-09-13_dna_r9.4.1_minion_promethion_384_ca963bcb
FLO-PRO002     SQK-LSK112-XL               dna_r9.4.1_e8.1_hac_prom       2021-09-13_dna_r9.4.1_minion_promethion_384_ca963bcb
FLO-PRO002     SQK-NBD112-24               dna_r9.4.1_e8.1_hac

In [17]:
#First Try. Did simplex calling for non-duplex reads which is not as accurate as the GuppyV5 'sup' calling. Going to use this for next run to not redo.
# Did not do 'call_non_duplex_reads' option so will only output duplex data. Want all data.
#bsub -J guppyM721 -o $WKDIR/log/guppyM721.out -e $WKDIR/log/guppyM721.err -R 'gpu' "guppy_duplex \
#--input_path squee \
#--save_path /workspace/hraijc/BB_M7_Assembly/GuppyV6 \
#--device 'cuda:0' \
#--do_read_splitting \
#--duplex_chunks_per_runner 1200"


Job <714006> is submitted to default queue <lowpriority>.


In [15]:
#copying output from step one from previous run so don't need to do again. 
#bsub -I "cp -r /workspace/hraijc/BB_M7_Assembly/GuppyV6/simplex/ /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex/"

In [6]:
#Second try. Skipping simplex (done before) and calling non_duplex_reads with GuppyV6 'sup' accuracy
#bsub -n 9 -J guppyM7224 -o $WKDIR/log/guppyM7224.out -e $WKDIR/log/guppyM7224.err -R 'select[defined(gpu)] rusage[mem=40000]' "guppy_duplex \
#--input_path /input/genomic/plant/Vaccinium/corymbosum/20220104_M7/20220104_1231_1A_PAI49175_9725852c/fast5_pass \
#--save_path /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex \
#--device 'cuda:0' \
#--do_read_splitting \
#--duplex_chunks_per_runner 1200 \
#--skip_simplex \
#--call_non_duplex_reads"


Job <715983> is submitted to default queue <lowpriority>.


In [7]:
#Third try. Was not able to skip simplex calling and read run1. Running again from scratch.
#Calling non_duplex_reads with GuppyV6 'sup' accuracy
bsub -n 9 -J guppyM7_3 -o $WKDIR/log/guppyM7_3.out -e $WKDIR/log/guppyM7_3.err -R 'select[defined(gpu)] rusage[mem=40000]' "guppy_duplex \
--input_path /input/genomic/plant/Vaccinium/corymbosum/20220104_M7/20220104_1231_1A_PAI49175_9725852c/fast5_pass \
--save_path /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3 \
--device 'cuda:0' \
--do_read_splitting \
--duplex_chunks_per_runner 1200 \
--call_non_duplex_reads"


Job <716298> is submitted to default queue <lowpriority>.


## Comparing quality of reads with Guppyv5 and Guppyv6 Duplex

### Using MinIONQC

In [5]:
#ONT QC
module load R/4.0.0
#Rscript /workspace/hraijc/git_clones/MinIONQC.R -h


Loading R/4.0.0
  Loading requirement: unixODBC/2.3.0 JAGS/4.2.0 gdal/2.4.0 proj/5.2.0


In [6]:
mkdir /workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV5
mkdir /workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV6_duplex
mkdir /workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV6_simplex
mkdir /workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV6_simplexduplex



mkdir: cannot create directory ‘/workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV5’: File exists


In [7]:
#QC GuppyV5
bsub -J minqc5  -o $WKDIR/log/minqc5.out -e $WKDIR/log/minqc5.err -n 1 \
"Rscript /workspace/hraijc/git_clones/MinIONQC.R -i /input/genomic/plant/Vaccinium/corymbosum/20220104_M7/20220104_1231_1A_PAI49175_9725852c/sequencing_summary_PAI49175_7e6a4738.txt -o $WKDIR/ONT_QC_GuppyV5 -c TRUE"



Job <725881> is submitted to default queue <lowpriority>.


In [17]:
#QC Guppy_duplex
bsub -J minqct  -o $WKDIR/log/minqct.out -e $WKDIR/log/minqct.err -n 1 \
"Rscript /workspace/hraijc/git_clones/MinIONQC.R -i /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/final/duplex/sequencing_summary.txt -o /workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV6_duplex -c TRUE"


Job <725898> is submitted to default queue <lowpriority>.


In [18]:
#QC Guppy_simplex
bsub -J minqct  -o $WKDIR/log/minqct.out -e $WKDIR/log/minqct.err -n 1 \
"Rscript /workspace/hraijc/git_clones/MinIONQC.R -i /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/final/simplex/sequencing_summary.txt -o /workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV6_simplex -c TRUE"



Job <725899> is submitted to default queue <lowpriority>.


In [6]:
bsub -I "cat /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/final/simplex/sequencing_summary.txt >> /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/sim_dup_seq_sum.txt"

Job <718590> is submitted to default queue <lowpriority>.
<<Waiting for dispatch ...>>
<<Starting on aklppg31>>


In [7]:
bsub -I "tail -n 88542 /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/final/duplex/sequencing_summary.txt >> /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/sim_dup_seq_sum.txt"

Job <718591> is submitted to default queue <lowpriority>.
<<Waiting for dispatch ...>>
<<Starting on aklppg31>>


In [9]:
bsub -I 'grep "filename" /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/sim_dup_seq_sum.txt'

Job <718593> is submitted to default queue <lowpriority>.
<<Waiting for dispatch ...>>
<<Starting on wkoppb41>>
filename	read_id	run_id	batch_id	channel	mux	start_time	duration	num_events	passes_filtering	template_start	num_events_template	template_duration	sequence_length_template	mean_qscore_template	strand_score_template	median_template	mad_template	scaling_median_template	scaling_mad_template


mkdir: cannot create directory ‘/workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV6_simplexduplex’: File exists


In [16]:
#QC Guppy_duplex+Guppy_Simplex
bsub -J minqct  -o $WKDIR/log/minqct.out -e $WKDIR/log/minqct.err -n 1 \
"Rscript /workspace/hraijc/git_clones/MinIONQC.R -i /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/sim_dup_seq_sum.txt -o /workspace/hraijc/BB_M7_Assembly/ONT_QC_GuppyV6_simplexduplex -c TRUE"


Job <725897> is submitted to default queue <lowpriority>.


### Using PycoQC

In [21]:
mkdir $WKDIR/PycoQC

In [3]:
ml load pycoqc

In [13]:
bsub -J pycoQC -o $WKDIR/log/pycoQC.out -e $WKDIR/log/pycoQC.err -n 1 \
"pycoQC -f /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/final/duplex/sequencing_summary.txt /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/final/simplex/sequencing_summary.txt \
    -o $WKDIR/PycoQC/BB_M7_ONTreport_GuppyV6Duplex.html \
    --report_title 'BB_M7_GuppyV6Duplex' \
    --min_pass_qual 10" 


Job <725925> is submitted to default queue <lowpriority>.


In [14]:
bsub -J pycoQC3 -o $WKDIR/log/pycoQC3.out -e $WKDIR/log/pycoQC3.err -n 1 \
"pycoQC -f /input/genomic/plant/Vaccinium/corymbosum/20220104_M7/20220104_1231_1A_PAI49175_9725852c/sequencing_summary_PAI49175_7e6a4738.txt -o $WKDIR/PycoQC/BB_M7_ONTreport_GuppyV5.html --report_title 'BB_M7_GuppyV5' --min_pass_qual 10" 


Job <725926> is submitted to default queue <lowpriority>.


In [15]:
bsub -J pycoQC2 -o $WKDIR/log/pycoQC2.out -e $WKDIR/log/pycoQC2.err -n 1 \
"pycoQC -f /workspace/hraijc/BB_M7_Assembly/GuppyV6_duplex_3/final/duplex/sequencing_summary.txt -o $WKDIR/PycoQC/BB_M7_ONTreport_GuppyV6_DuplexONLY.html --report_title 'BB_M7_GuppyV6_DuplexONLY' --min_pass_qual 10" 


Job <725927> is submitted to default queue <lowpriority>.
